### Beginning DuckDB  
In this post, I am documenting my learning about DuckDB.   

DuckDB is a relational database that supports the SQL language. Think about your common database solution such as MySQL, etc. However, it lives mostly in memory. No need to connect to a remote destination IP and port. If working with data already written to the disk, then just point duckdb to the **.duckdb** file.

DuckDB is considered a columnar database. While most of us may work with Pandas dataframe directly, DuckDB allows us to run SQL queries against those Pandas DataFrame if needed similarly to what you would have done against a remote SQL database.  

We are able to run queries on data whether that data is on our locally or remotely. 

DuckDB is very useful in environments where there is not much memory  


In [2]:
# First up, let's install DuckDB
!pip install duckdb --upgrade --verbose --no-cache-dir

Using pip 24.2 from /home/securitynik/miniconda3/envs/main_playground/lib/python3.12/site-packages/pip (python 3.12)


In [3]:
# With DuckDB installed, let's import the library
import duckdb

In [4]:
# Let's get the duckdb version
duckdb.__version__

'1.1.3'

In [5]:
# Or alternatively
duckdb.sql(query='CALL pragma_version()')

┌─────────────────┬────────────┐
│ library_version │ source_id  │
│     varchar     │  varchar   │
├─────────────────┼────────────┤
│ v1.1.3          │ 19864453f7 │
└─────────────────┴────────────┘

In [6]:
# What platform am I running duckdb on
duckdb.sql(query='CALL pragma_platform()')

┌──────────────────┐
│     platform     │
│     varchar      │
├──────────────────┤
│ linux_amd64_gcc4 │
└──────────────────┘

In [7]:
# Get the user agent
duckdb.sql(query='PRAGMA user_agent')

┌────────────────────────────────────────────────┐
│                   user_agent                   │
│                    varchar                     │
├────────────────────────────────────────────────┤
│ duckdb/v1.1.3(linux_amd64_gcc4) python jupyter │
└────────────────────────────────────────────────┘

In [8]:
# Get the current DuckDB settings
duckdb.sql(query='SELECT * FROM duckdb_settings()')

┌─────────────────────────────────┬─────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────┐
│              name               │      value      │                                                                                                  description                                                                                                  │ input_type │  scope  │
│             varchar             │     varchar     │                                                                                                    varchar                                                                                                    │  varchar   │ varchar │
├─────────────────────────────────┼─────────────────┼────────────────────────────────────────────────────────────────────────────────────────────

In [9]:
# Set the logging
duckdb.sql(query='SET log_query_path = "/tmp/duckdb.log"')
!ls /tmp/duckdb.log

/tmp/duckdb.log


In [10]:
# Get a list of the available functions
duckdb.sql('PRAGMA functions')

┌────────────┬─────────┬────────────────────────┬─────────┬─────────────┬──────────────┐
│    name    │  type   │       parameters       │ varargs │ return_type │ side_effects │
│  varchar   │ varchar │       varchar[]        │ varchar │   varchar   │   boolean    │
├────────────┼─────────┼────────────────────────┼─────────┼─────────────┼──────────────┤
│ !__postfix │ SCALAR  │ [INTEGER]              │ NULL    │ HUGEINT     │ false        │
│ !~~        │ SCALAR  │ [VARCHAR, VARCHAR]     │ NULL    │ BOOLEAN     │ false        │
│ !~~*       │ SCALAR  │ [VARCHAR, VARCHAR]     │ NULL    │ BOOLEAN     │ false        │
│ %          │ SCALAR  │ [UHUGEINT, UHUGEINT]   │ NULL    │ UHUGEINT    │ false        │
│ %          │ SCALAR  │ [UBIGINT, UBIGINT]     │ NULL    │ UBIGINT     │ false        │
│ %          │ SCALAR  │ [UINTEGER, UINTEGER]   │ NULL    │ UINTEGER    │ false        │
│ %          │ SCALAR  │ [USMALLINT, USMALLINT] │ NULL    │ USMALLINT   │ false        │
│ %          │ SCALAR

In [11]:
# Take it for a test spin
duckdb.sql('SELECT 1').show()

┌───────┐
│   1   │
│ int32 │
├───────┤
│     1 │
└───────┘



With that simple test out of the way, let's get some real data to work with.  

DuckDB is capable of reading data from different formats, such as CSV, Parquet, JSON, Pandas DataFrame, Numpy arrays,  etc. It can read CSV files using the **.read_csv** function or even directly via **.sql**. This **.sql** allows DuckDB to operate on in memory databases.To access an in memory database use the **connect** method without any arguments.


While your CSV, JSON, etc., files are generally stored on your system, there is nothing stopping you from downloading from a remote location via https. If the need arises for you to store your data to disk, as in make it persistent, DuckDB also has facilities for that.  

Let us get some data. For us, let's read the conn.log file from Zeek. This file is serialized in JSON format.

In [12]:
# Read the data and take a quick peak
quack = duckdb.read_json(path_or_buffer='/mnt/d/ML/conn-today.json')
quack

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history  │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │        duration        │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar  │   int64   │     int64     │   int64   │     int64     │         double         │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼──────

Now that we know we can read data, let's write this data back to the file system for testing. In this case, let's write in the Parquet format.

In [13]:
# Writing to the file system
quack.write_parquet(file_name=r'/tmp/conn-today.pqt')

# With the above done, let's verify the file was successfully written
!ls -l /mnt/d/ML/conn-today.pqt

-rwxrwxrwx 1 securitynik securitynik 1006239 Jan  1 13:54 /mnt/d/ML/conn-today.pqt


We now know we can open files and write to the file system. Let's create persistent database. That is a database that is stored to the file system.  

In [14]:
# Create the database
duckdb.connect(database='/tmp/securitynik.duckdb', read_only=False)

# Verify the file was created
!ls -l /tmp/securitynik.duckdb

-rw-r--r-- 1 securitynik securitynik 12288 Jan  6 19:42 /tmp/securitynik.duckdb


In [15]:
# Let's actually connect to that database and do some work with it
con = duckdb.connect(database='/tmp/securitynik.duckdb', read_only=False)
con

In [16]:
# With the connection in place, time to create a table
# Let's build a table using the information we stored in the database earlier
# If the table exists, then just move on
con.sql('CREATE TABLE IF NOT EXISTS conn_log AS FROM read_parquet("/tmp/conn-today.pqt")')

In [17]:
# Look at the list of databases and their paths
# We created one so the expectation here is only one
con.sql('PRAGMA database_list')

┌───────┬─────────────┬─────────────────────────┐
│  seq  │    name     │          file           │
│ int64 │   varchar   │         varchar         │
├───────┼─────────────┼─────────────────────────┤
│  1146 │ securitynik │ /tmp/securitynik.duckdb │
└───────┴─────────────┴─────────────────────────┘

In [18]:
# Get information on the database size
con.sql('CALL pragma_database_size()')

┌───────────────┬───────────────┬────────────┬──────────────┬─────────────┬─────────────┬──────────┬──────────────┬──────────────┐
│ database_name │ database_size │ block_size │ total_blocks │ used_blocks │ free_blocks │ wal_size │ memory_usage │ memory_limit │
│    varchar    │    varchar    │   int64    │    int64     │    int64    │    int64    │ varchar  │   varchar    │   varchar    │
├───────────────┼───────────────┼────────────┼──────────────┼─────────────┼─────────────┼──────────┼──────────────┼──────────────┤
│ securitynik   │ 0 bytes       │     262144 │            0 │           0 │           0 │ 3.3 MiB  │ 11.1 MiB     │ 12.3 GiB     │
└───────────────┴───────────────┴────────────┴──────────────┴─────────────┴─────────────┴──────────┴──────────────┴──────────────┘

In [19]:
# How about we change the amount for the memory limit of the database
con.sql(query="SET memory_limit = '1GB'")
con.sql('CALL pragma_database_size()')

┌───────────────┬───────────────┬────────────┬──────────────┬─────────────┬─────────────┬──────────┬──────────────┬──────────────┐
│ database_name │ database_size │ block_size │ total_blocks │ used_blocks │ free_blocks │ wal_size │ memory_usage │ memory_limit │
│    varchar    │    varchar    │   int64    │    int64     │    int64    │    int64    │ varchar  │   varchar    │   varchar    │
├───────────────┼───────────────┼────────────┼──────────────┼─────────────┼─────────────┼──────────┼──────────────┼──────────────┤
│ securitynik   │ 0 bytes       │     262144 │            0 │           0 │           0 │ 3.3 MiB  │ 11.1 MiB     │ 953.6 MiB    │
└───────────────┴───────────────┴────────────┴──────────────┴─────────────┴─────────────┴──────────┴──────────────┴──────────────┘

In [20]:
# Let's list all tables in the securitynik database
con.sql('PRAGMA show_tables')

┌──────────┐
│   name   │
│ varchar  │
├──────────┤
│ conn_log │
└──────────┘

In [21]:
# Expand that view
# Let's get a little bit more data about the tables in database, tables, columns, etc.
con.sql('PRAGMA show_tables_expanded')

┌─────────────┬─────────┬──────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name   │                                                                                                      column_names                                                                                                      │                                                                               column_types                                                                                │ temporary │
│   varchar   │ varchar │ varchar  │                                                                                        

In [22]:
# Describe the table
con.sql('DESCRIBE conn_log')

┌───────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│  column_name  │ column_type │  null   │   key   │ default │  extra  │
│    varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ts            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ uid           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.orig_h     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.orig_p     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ id.resp_h     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ id.resp_p     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ proto         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ conn_state    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ local_orig    │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ local_resp    │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NU

In [23]:
# Get information on the table from a different perspective
con.sql('PRAGMA table_info(conn_log)')

┌───────┬───────────────┬─────────┬─────────┬────────────┬─────────┐
│  cid  │     name      │  type   │ notnull │ dflt_value │   pk    │
│ int32 │    varchar    │ varchar │ boolean │  varchar   │ boolean │
├───────┼───────────────┼─────────┼─────────┼────────────┼─────────┤
│     0 │ ts            │ DOUBLE  │ false   │ NULL       │ false   │
│     1 │ uid           │ VARCHAR │ false   │ NULL       │ false   │
│     2 │ id.orig_h     │ VARCHAR │ false   │ NULL       │ false   │
│     3 │ id.orig_p     │ BIGINT  │ false   │ NULL       │ false   │
│     4 │ id.resp_h     │ VARCHAR │ false   │ NULL       │ false   │
│     5 │ id.resp_p     │ BIGINT  │ false   │ NULL       │ false   │
│     6 │ proto         │ VARCHAR │ false   │ NULL       │ false   │
│     7 │ conn_state    │ VARCHAR │ false   │ NULL       │ false   │
│     8 │ local_orig    │ BOOLEAN │ false   │ NULL       │ false   │
│     9 │ local_resp    │ BOOLEAN │ false   │ NULL       │ false   │
│    10 │ missed_bytes  │ BIGINT  

In [24]:
# Get the storage info for the table
con.sql(query='CALL pragma_storage_info(conn_log)')

┌──────────────┬─────────────┬───────────┬─────────────┬────────────┬──────────────┬───────┬───────┬──────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────┬────────────┬──────────┬──────────────┬──────────────┐
│ row_group_id │ column_name │ column_id │ column_path │ segment_id │ segment_type │ start │ count │ compression  │                                                     stats                                                     │ has_updates │ persistent │ block_id │ block_offset │ segment_info │
│    int64     │   varchar   │   int64   │   varchar   │   int64    │   varchar    │ int64 │ int64 │   varchar    │                                                    varchar                                                    │   boolean   │  boolean   │  int64   │    int64     │   varchar    │
├──────────────┼─────────────┼───────────┼─────────────┼────────────┼──────────────┼───────┼───────┼────────────

In [25]:
# Let's now confirm the data can be accessed
con.sql('SELECT * FROM conn_log')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history  │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │        duration        │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar  │   int64   │     int64     │   int64   │     int64     │         double         │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼──────

In [26]:
# Try a different command
# Count the number fo records
con.sql('SELECT COUNT(*) FROM conn_log')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        24048 │
└──────────────┘

In [27]:
# try the execute command to grab one record
con.execute('SELECT * FROM conn_log')
print(con.fetchone())

(1696017594.59247, 'ChefyE3DEW1aClvrpg', '127.0.0.1', 6789, '127.0.0.1', 59106, 'tcp', 'OTH', True, True, 0, 'Ccc', 0, 0, 0, 0, None, None, None, None)


**Getting our SQL Kung-fu on**

In [28]:
# Let's select the first 10 records and order them by the timestamp (ts) column
con.sql('SELECT * FROM conn_log ORDER BY ts LIMIT 10')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬─────────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────┬────────────┬────────────┬─────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │   history   │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │      duration      │ orig_bytes │ resp_bytes │ service │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │   varchar   │   int64   │     int64     │   int64   │     int64     │       double       │   int64    │   int64    │ varchar │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────────────┼─────────

In [29]:
# How many unique source IPs. are there
con.sql('SELECT DISTINCT COUNT("id.orig_h") FROM conn_log')

┌────────────────────┐
│ count("id.orig_h") │
│       int64        │
├────────────────────┤
│              24048 │
└────────────────────┘

In [30]:
# Get 10 unique source IP addresses
con.sql('SELECT DISTINCT "id.orig_h" FROM conn_log ORDER BY ts LIMIT 10')

┌───────────────────────────┐
│         id.orig_h         │
│          varchar          │
├───────────────────────────┤
│ 192.168.0.62              │
│ 192.168.0.75              │
│ 192.168.0.24              │
│ 192.168.0.31              │
│ fe80::eb3f:74b2:4679:6bae │
│ fe80::8450:9cff:fe16:ef95 │
│ 192.168.0.2               │
│ 192.168.0.78              │
│ 192.168.0.8               │
│ 144.195.33.213            │
├───────────────────────────┤
│          10 rows          │
└───────────────────────────┘

In [31]:
# Get the largest duration value from the conn_log table
con.sql('SELECT MAX(duration) FROM conn_log')

┌────────────────────┐
│   max(duration)    │
│       double       │
├────────────────────┤
│ 115607.98462510109 │
└────────────────────┘

In [32]:
# Get a sum of the duration
# Get the largest duration
con.sql('SELECT SUM(duration) FROM conn_log')

┌───────────────────┐
│   sum(duration)   │
│      double       │
├───────────────────┤
│ 825393.7917554379 │
└───────────────────┘

In [33]:
# Find all records where the duration is greater than 10 minutes
con.sql('SELECT * FROM conn_log WHERE duration > 600')

┌───────────────────┬────────────────────┬──────────────┬───────────┬────────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬──────────────────┬───────────┬───────────────┬───────────┬───────────────┬────────────────────┬────────────┬────────────┬──────────┐
│        ts         │        uid         │  id.orig_h   │ id.orig_p │   id.resp_h    │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │     history      │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │      duration      │ orig_bytes │ resp_bytes │ service  │
│      double       │      varchar       │   varchar    │   int64   │    varchar     │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │     varchar      │   int64   │     int64     │   int64   │     int64     │       double       │   int64    │   int64    │ varchar  │
├───────────────────┼────────────────────┼──────────────┼───────────┼────────────────┼───────────┼─────────┼────

In [34]:
# Summarize the table
con.sql(query='SUMMARIZE conn_log')

┌───────────────┬─────────────┬─────────────────────┬───────────────────────────┬───────────────┬────────────────────┬────────────────────┬───────────────────────┬────────────────────────┬─────────────────────┬───────┬─────────────────┐
│  column_name  │ column_type │         min         │            max            │ approx_unique │        avg         │        std         │          q25          │          q50           │         q75         │ count │ null_percentage │
│    varchar    │   varchar   │       varchar       │          varchar          │     int64     │      varchar       │      varchar       │        varchar        │        varchar         │       varchar       │ int64 │  decimal(9,2)   │
├───────────────┼─────────────┼─────────────────────┼───────────────────────────┼───────────────┼────────────────────┼────────────────────┼───────────────────────┼────────────────────────┼─────────────────────┼───────┼─────────────────┤
│ ts            │ DOUBLE      │ 1695902349.570724   

In [35]:
# We might instead want to see the information above as a dataframe
# Let's use the **.df** function to get the data as a dataframe
con.sql('SELECT "id.orig_h", "id.resp_h" FROM conn_log WHERE duration > 600').df()

,id.orig_h,id.resp_h
0,192.168.0.31,165.225.208.44
1,192.168.0.31,165.225.208.44
2,192.168.0.31,165.225.208.44
3,192.168.0.95,34.98.105.85
4,192.168.0.95,34.149.46.130
...,...,...
112,192.168.0.31,192.168.0.4
113,192.168.0.31,192.168.0.4
114,192.168.0.31,192.168.0.4
115,192.168.0.28,52.236.189.131


In [36]:
# Let pull back all the tables and some rows and store these into a dataframe
tmp_df = con.sql('SELECT * FROM conn_log WHERE duration > 600').df()
tmp_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,duration,orig_bytes,resp_bytes,service
0,1.696001e+09,CthQmt3D5lmDwrRxUl,192.168.0.31,64113,165.225.208.44,443,tcp,RSTR,True,False,0,ShADadttfr,821,57635,1113,77717,16416.052616,24783,32633,"http,ssl"
1,1.695988e+09,CYUQe22Chebdk2ue1e,192.168.0.31,51760,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdaTtr,17442,8006593,15058,1295176,29656.265580,7308515,692511,"http,ssl"
2,1.695994e+09,CRqfCIG05qUCcFcF,192.168.0.31,55543,165.225.208.44,443,tcp,RSTR,True,False,0,ShADdafr,3150,757605,1603,212389,23924.337346,631593,148257,"http,ssl"
3,1.696017e+09,CfLKAp3pg8ZlCxVyk,192.168.0.95,35630,34.98.105.85,443,tcp,SF,True,False,0,ShADadtFf,20,2284,22,3070,618.818055,1212,1772,ssl
4,1.696017e+09,CSRhux2BwyF9SSzjDf,192.168.0.95,49344,34.149.46.130,443,tcp,SF,True,False,0,ShADadtFf,18,1960,15,6040,610.352108,1004,5213,ssl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,1.695988e+09,CsqLYG90S5u8wUGji,192.168.0.31,55906,192.168.0.4,9200,tcp,OTH,True,True,0,DacAcTcTcT,37698,42754618,9589,383560,29664.822755,41183458,3923632,None
113,1.695988e+09,C3uhyF2oaouFzSjN97,192.168.0.31,56024,192.168.0.4,9200,tcp,OTH,True,True,0,DacAccc,32628,40508275,5091,203640,29650.207311,39203107,4989212,None
114,1.695988e+09,CaXyts2pErtFF9wXa,192.168.0.31,64349,192.168.0.4,8220,tcp,OTH,True,True,0,DcTcTAacTcT,2372,503027,362,14480,29651.999841,406240,25806,None
115,1.696018e+09,CiIl0H14bcFsvU5rl8,192.168.0.28,62619,52.236.189.131,443,tcp,SF,True,False,0,ShADadFRf,36,3259,23,5250,2881.177577,1807,4318,ssl


In [37]:
# Earlier above, we said these dataframes can be queried as if they were SQL tables. Let us verify that
# Finding records that are not TCP traffic
duckdb.sql("SELECT * FROM tmp_df WHERE proto != 'tcp' ")

┌──────────────────┬────────────────────┬─────────────┬───────────┬─────────────┬───────────┬─────────┬────────────┬────────────┬────────────┬──────────────┬─────────┬───────────┬───────────────┬───────────┬───────────────┬───────────────────┬────────────┬────────────┬─────────┐
│        ts        │        uid         │  id.orig_h  │ id.orig_p │  id.resp_h  │ id.resp_p │  proto  │ conn_state │ local_orig │ local_resp │ missed_bytes │ history │ orig_pkts │ orig_ip_bytes │ resp_pkts │ resp_ip_bytes │     duration      │ orig_bytes │ resp_bytes │ service │
│      double      │      varchar       │   varchar   │   int64   │   varchar   │   int64   │ varchar │  varchar   │  boolean   │  boolean   │    int64     │ varchar │   int64   │     int64     │   int64   │     int64     │      double       │   int64    │   int64    │ varchar │
├──────────────────┼────────────────────┼─────────────┼───────────┼─────────────┼───────────┼─────────┼────────────┼────────────┼────────────┼──────────────┼───

In [38]:
# Let us read a file from remote 
# Define the Iris URL
iris_con = duckdb.sql("SELECT * FROM 'https://raw.githubusercontent.com/SecurityNik/Data-Science-and-ML/refs/heads/main/DATASETS/Iris.csv', LIMIT 10")
iris_con

┌───────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────────┐
│  Id   │ SepalLengthCm │ SepalWidthCm │ PetalLengthCm │ PetalWidthCm │   Species   │
│ int64 │    double     │    double    │    double     │    double    │   varchar   │
├───────┼───────────────┼──────────────┼───────────────┼──────────────┼─────────────┤
│     1 │           5.1 │          3.5 │           1.4 │          0.2 │ Iris-setosa │
│     2 │           4.9 │          3.0 │           1.4 │          0.2 │ Iris-setosa │
│     3 │           4.7 │          3.2 │           1.3 │          0.2 │ Iris-setosa │
│     4 │           4.6 │          3.1 │           1.5 │          0.2 │ Iris-setosa │
│     5 │           5.0 │          3.6 │           1.4 │          0.2 │ Iris-setosa │
│     6 │           5.4 │          3.9 │           1.7 │          0.4 │ Iris-setosa │
│     7 │           4.6 │          3.4 │           1.4 │          0.3 │ Iris-setosa │
│     8 │           5.0 │          3.4 │           1.5

In [39]:
# There was no need to download the file first. 
# just extract two columns from the dataframe just to see what can be done
iris_con.to_df()[['SepalLengthCm', 'Species']]


,SepalLengthCm,Species
0,5.1,Iris-setosa
1,4.9,Iris-setosa
2,4.7,Iris-setosa
3,4.6,Iris-setosa
4,5.0,Iris-setosa
5,5.4,Iris-setosa
6,4.6,Iris-setosa
7,5.0,Iris-setosa
8,4.4,Iris-setosa
9,4.9,Iris-setosa


In [40]:
# Make a connection again. This time by setting up the database
iris_con = duckdb.connect() 

# Store the results in a table
iris_con.sql("SELECT * FROM 'https://raw.githubusercontent.com/SecurityNik/Data-Science-and-ML/refs/heads/main/DATASETS/Iris.csv', LIMIT 10").to_table('iris_table')

# Present that table
iris_con.table('iris_table').df()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [41]:
print(iris_con.sql(query='EXPLAIN (FORMAT HTML) SELECT SepalLengthCm FROM iris_table'))

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [45]:
# Well if we opened a connection to a table, we should close it too
iris_con.close()

*References*    
- https://duckdb.org/docs/installation/index?version=stable&environment=python  
- https://duckdb.org/2024/03/29/external-aggregation  
- https://duckdb.org/docs/api/python/data_ingestion.html  
- https://duckdb.org/docs/sql/statements/create_table.html  
- https://duckdb.org/docs/sql/statements/create_table.html  
- https://duckdb.org/why_duckdb
- https://motherduck.com/blog/duckdb-ecosystem-newsletter-november-2024/  
- https://duckdb.org/2024/10/04/duckdb-user-survey-analysis.html  
- https://practicaldataengineering.substack.com/p/building-data-pipeline-using-duckdb  
- https://practicaldataengineering.substack.com/p/duckdb-beyond-the-hype  
- https://duckdb.org/pdf/SIGMOD2019-demo-duckdb.pdf   
- https://medium.com/@bruhhmx/introduction-to-duckdb-7d0b4c0de7f5  
- https://r.duckdb.org/reference/duckdb_register.html  
- https://duckdb.org/docs/configuration/pragmas.html  
- https://duckdb.org/docs/configuration/overview.html   
- https://duckdb.org/docs/guides/overview.html  
- https://duckdb.org/2024/09/25/changing-data-with-confidence-and-acid.html  
- https://maximebeauchemin.medium.com/functional-data-engineering-a-modern-paradigm-for-batch-data-processing-2327ec32c42a  
- https://duckdb.org/docs/api/cli/dot_commands  
- https://duckdb.org/2024/03/29/external-aggregation  
- https://duckdb.org/2022/03/07/aggregate-hashtable.html  
- https://shekhargulati.com/2019/12/15/the-5-minute-introduction-to-duckdb-the-sqlite-for-analytics/
- https://open.spotify.com/episode/7mp1QYRJR5Q8Hg9gHACVGG  